In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))

## 读取配置文件

In [ ]:
import yaml
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)

## Dataset加载与格式化

In [ ]:
from utils import *
from dataset import Dataset
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()
print(validation_dataset[2])  

if not isinstance(train_dataset, list):
        logging.info('Train dataset: %s', train_dataset)

answerable_indices, unanswerable_indices = split_dataset(train_dataset)


if config["dataset"]['answerable_only']:
        unanswerable_indices = []
        val_answerable, val_unanswerable = split_dataset(validation_dataset)
        del val_unanswerable
        validation_dataset = [validation_dataset[i] for i in val_answerable]

## Prompt Engineer

In [ ]:
from prompt_engineer import PromptGenerator

promptgenerator = PromptGenerator(config,validation_dataset)

In [ ]:
prompt = promptgenerator.generate_template_prompt_by_id(1)
print(prompt)
print(len(promptgenerator.get_unused_indices()))


In [ ]:
prompts = promptgenerator.generate_template_prompts_by_count(2)
print(prompt)
print(len(promptgenerator.get_unused_indices()))

In [ ]:
import random
indices = random.sample(promptgenerator.get_unused_indices(),2)
print(indices)
prompts =promptgenerator.generate_template_prompts_from_indices(indices)
len(promptgenerator.get_unused_indices())

In [ ]:
few_shot_prompt = promptgenerator.construct_fewshot_prompt_from_indices()
print(few_shot_prompt)
print(len(promptgenerator.get_unused_indices()))

## model

In [ ]:
from models import HuggingfaceModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
huggingface_model = HuggingfaceModel(config)

In [ ]:
#temperature=1.0,return_full=False
output_text = huggingface_model.predict(prompt)
print("Generated Text > \n", output_text)

In [ ]:
import math

user_tag = "USER:"
assistant_tag = "ASSISTANT:"
input = "What is the capital of France?"
v = huggingface_model.get_p_true(input,"Washington")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Paris")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Beijing")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

v = huggingface_model.get_p_true(input,"\n")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

In [ ]:
input_text = prompts[1]

output_text,log = huggingface_model.predict(input_text)
print("<Generated Text>:", output_text)
print(log)